In [2]:
import tkinter as tk
import numpy as np
import time
import random
import copy
import math

# Set number of rows and columns
ROWS = 50
COLS = 50
ANIMATION_DELAY = 20
NEIGHBOUR_SEARCH_DELAY = 0
BORDER_PROB = 0.2
ITERATIONS_CONWAY = 1000

# Create a grid of None to store the references to the tiles
cells = [[None for _ in range(ROWS)] for _ in range(COLS)]

class Cell:

    def __init__(self, col, row):
        self.col = col
        self.row = row
        self.rect = None
        self.text = None
        createRectangle(self,"","","")
        # extras for A_Star:
        self.cost_g = 0
        self.cost = 0
        self.parent = None
        self.dist_to_neighbours = []
    
    def equals(self,cell):
        return (self.col == cell.col and self.row == cell.row)
    
    def changeText(self,txt,color):
        c.itemconfig(self.text,text=txt,fill=color)
        
    def changeRect(self,color):
        c.itemconfig(self.rect, fill=color)
    
    def toString(self):
        return str(self.col) + "|" + str(self.row)
    
    # extras for A_Star:
    def getDist(self,cell):
        return 1

    
def callback(event):
    # Calculate column and row number
    col = int(event.x/col_width)
    row = int(event.y/row_height)
    clicked_cell = cells[col][row]
    # If the tile is not filled, create a rectangle
    c.itemconfig(cells[clicked_cell.col][clicked_cell.row].rect,fill="black")
    c.itemconfig(cells[4][4].rect,fill="red")
    start = clicked_cell
    goal = cells[4][4]
    
    #BFS(start,goal,True)
    #DFS(start,goal,True)
    a = A_Star(start,goal)

def getNeighbours(coord):
    neighbours = []
    for row in range(3):
        for col in range(3):
            new_col = (coord.col-1 + col) % COLS
            new_row = (coord.row-1 + row) % ROWS
            
            isStartingCell = (row == 1 and col == 1)
            isDiagonal     = not isStartingCell and ((col + row) % 2 == 0)
            isWithinMap    = (new_col >= 0 and new_row >= 0 and new_row < ROWS and new_col < COLS)
            isBorder       = isWithinMap and c.itemcget(cells[new_col][new_row].rect,"fill") == "black"
            
            if not isStartingCell and not isDiagonal and isWithinMap and not isBorder:
                neighbours.append(cells[new_col][new_row])
                #c.create_rectangle(upper_left.col + col * col_width, upper_left.row + row * row_height, (upper_left.col + col+1)*col_width, (upper_left.row + row+1)*row_height, fill="blue")
    #print(len(neighbours))
    return neighbours

def createRectangle(cell,rect_color,txt,text_color):
        cell.rect = c.create_rectangle(cell.col*col_width, cell.row*row_height, (cell.col+1)*col_width, (cell.row+1)*row_height, fill=rect_color)
        cell.text = c.create_text(((cell.col*col_width + (cell.col+1)*col_width)/2, (cell.row*row_height + (cell.row+1)*row_height)/2), text=txt, fill=text_color)
        #c.after(1000, createRectangle, coord,rect_color,txt,text_color)

def searchNeighbours(goal,neighbours,rect_color,txt,text_color,delay=NEIGHBOUR_SEARCH_DELAY/1000):
    for neighbour in neighbours:
        #neighbour.changeText(txt, text_color)
        if not neighbour.equals(goal):
            neighbour.changeRect(rect_color) 
        time.sleep(delay)
        c.update()
        
def extractAlreadySearched(neighbours, explored, frontier):
    search = []
    for neighbour in neighbours:
        if neighbour not in explored and neighbour not in frontier:
            search.append(neighbour)
    return search
    
def BFS(start,goal,randomized):
    frontier = [start]
    explored = []
    
    while not frontier==[]:
        next_cell = frontier.pop(0)
        explored.append(next_cell)
        next_cell.changeRect("green")
        if next_cell.equals(goal):
            print("Goal found!")
            return
        neighbours = getNeighbours(next_cell)
        search = extractAlreadySearched(neighbours,frontier, explored)
        if randomized:
            random.shuffle(search)
        frontier.extend(search)
        time.sleep(ANIMATION_DELAY/1000);
        c.update()
        if not len(search)==0:
            searchNeighbours(goal,search,"blue","added to \n frontier","white")
            
    print("Goal not found!")
            
def DFS(start,goal,randomized):
    frontier = [start]
    explored = []
    
    while not frontier==[]:
        next_cell = frontier.pop(0)
        explored.append(next_cell)
        next_cell.changeRect("green")
        if next_cell.equals(goal):
            print("Goal found!")
            return None
        neighbours = getNeighbours(next_cell)
        search = extractAlreadySearched(neighbours,frontier, explored)
        if randomized:
            random.shuffle(search)
        frontier = search + frontier
        time.sleep(ANIMATION_DELAY/1000);
        c.update()
        if not len(search)==0:
            searchNeighbours(goal,search,"blue","added to \n frontier","white")
    
    print("Goal not found!")
            
def createRandomMaze():
    maze = (np.random.random((COLS,ROWS)) < BORDER_PROB) *1
    for row in range(ROWS):
            for col in range(COLS):
                isBorder = maze[col][row]
                if isBorder:
                    cells[col][row].changeRect("black")

def initializeCanvas(hasBorder):
    for row in range(ROWS):
        for col in range(COLS):
            cells[col][row] = Cell(col,row)
            if hasBorder and (row == 0 or col == 0 or col == COLS-1 or row == ROWS-1):
                cells[col][row].changeRect("black")
                
class A_Star:

    def __init__(self,initialCity, goalCity):
        currentCity = copy.deepcopy(initialCity)
        currentCity.cost_g = 0
        currentCity.cost = self.heuristic(initialCity, goalCity) + currentCity.cost_g

        # --------------- changed, as it did not recognize frontier as a list, when currentCity was added
        # like this: frontier = [currentCity]
        frontier = []
        frontier.append(currentCity)
        # -----------------------------------------

        explored = []
        [explored, frontier, currentCity] = self.reachGoalNode(currentCity, explored, frontier, goalCity)

        if not currentCity is None:
            # why do we need the explored set right here? because we have saved the parents and because the chosen
            # heuristic is consistent, we can find the path by going through the parent of the goalcity and the 
            # parents following the goalCity
            [path, cheapest_cost] =  self.findPath(explored, initialCity, currentCity,path=[currentCity])
            print("Goal found!")
            print('The shortest path is the yellow path.')
            for x in path:
                x.changeRect("yellow")
            path[-1].changeRect("red")
            print('The path has a total cost of:',cheapest_cost)
            #return path, cheapest_cost
        else:
            print("Goal not found!")
        
        self.initializeAllCities();

    def heuristic(self,cell1, cell2):
        # Remove the next line
        x1 = cell1.col
        y1 = cell1.row

        x2 = cell2.col
        y2 = cell2.row

        dist = math.sqrt((x2 - x1)**2 + (y2-y1)**2)

        return dist
    
    def updateFrontier(self,currentCity = None, frontier = [], explored = [], goalCity = None):
    
        #preparing the node that will be expanded next
        frontier.remove(currentCity)
        explored.append(currentCity)
        
        currentCity.changeRect("green")
        time.sleep(ANIMATION_DELAY/1000);
        c.update()

        # searching through all of its neighbors
        for neighbour in getNeighbours(currentCity):

            # there is no need to find a path ging back to the starting point, as the shortest path
            # to it is the node itself
            if neighbour.equals(explored[0]):
                continue

            # if neighbor ist not in the explored set, it can be either added
            # to the frontier or have its values in the frontier updated
            if not neighbour in explored:
                # calculating the new g(n)
                newCost_g = currentCity.cost_g + currentCity.getDist(neighbour)
                # calculating the new f(n)
                newCost = newCost_g + self.heuristic(neighbour, goalCity)
                # nodes and paths are only updated when the corresponding costs are lower then the previous ones
                # neighbour.cost == 0 is needed because the cities are initialized with cost = 0 at the beginning
                if newCost < neighbour.cost or neighbour.cost == 0:
                    neighbour.parent = currentCity
                    neighbour.cost_g = newCost_g 
                    neighbour.cost = newCost
               #     print('go to ' + goalCity.name + ' from ' + neighbour.name + ' via ' + currentCity.name + ' with est. cost: ' + str(neighbour.cost))
                if not neighbour in frontier:
                    neighbour.changeRect("blue")
                    time.sleep(ANIMATION_DELAY/1000);
                    c.update()
                    frontier.append(neighbour)

      #  print()
        return frontier, explored

    def chooseNode(self,frontier):
        if frontier == []:
            currentCity = None
        else:
            # assume that the first element of th frontier is the one with the minimal cost
            currentCity = frontier[0]
            minCost = frontier[0].cost
            minIndex = 0

            # look for the city with the current minimal cost
            for i in range(1, len(frontier)):
                if frontier[i].cost < minCost:
                    minCost = frontier[i].cost
                    minIndex = i
                    currentCity = frontier[minIndex]
        return currentCity
    
    def initializeAllCities(self):
        for row in range(ROWS):
            for col in range(COLS):
                cells[col][row].cost = 0
                cells[col][row].cost_g = 0
                cells[col][row].parent = None
    
    def reachGoalNode(self,currentCity, explored, frontier, goalCity):
    
        # if the city is the next node to be expanded, we can stop because our heuristic is consistent
        # (known fact in lecture), meaning that every time a node is chosen for expansion, the shortest path
        # to it has been found and saved via saving all corresponding parents
        if currentCity is None:
            return explored, frontier, currentCity
        
        if currentCity.equals(goalCity):
            explored.append(currentCity)
            return explored, frontier, currentCity
        else:
            self.updateFrontier(currentCity, frontier, explored, goalCity)
            return self.reachGoalNode(self.chooseNode(frontier), explored, frontier, goalCity)
    
    def findPath(self,explored, initialCity, currentCity, cheapest_cost=0, path=[]):
    
        # we are recursively collecting all of the parents, so we stop when we reach the 
        # start position
        if currentCity.equals(initialCity):
            return path, cheapest_cost
        else:
            # set the previous city, add the distance to the costs and expand the path
            previousCity = currentCity.parent
            cheapest_cost = cheapest_cost + previousCity.getDist(currentCity)
            path = [previousCity] + path 
            return self.findPath(explored, initialCity, previousCity, cheapest_cost, path)
    
    
            
    

test = 0;


    

# Create the window, a canvas and the mouse click event binding
root = tk.Tk()
c = tk.Canvas(root, width=500, height=500, borderwidth=5, background='white')
c.pack()
c.bind("<Button-1>", callback)
#c.bind("<Button-1>", callback2)
c.update()
# Get rectangle diameters
col_width = c.winfo_width()/COLS
row_height = c.winfo_height()/ROWS
initializeCanvas(True)

#print([x.toString() for x in getAliveNeighbours(cells[0][0])])

createRandomMaze()
#n = getNeighbours(cells[2][2])
#for x in n:
#    x.toString()
            
root.mainloop()

Goal found!
The shortest path is the yellow path.
The path has a total cost of: 57
